In [ ]:
"""
THE MOONLIT SHRINE — a tiny parser adventure
- ≥3 Locations (we have 6)
- ≥3 Items (we have rope, torch, flint, iron key, moon idol)
- Puzzles:
    * Light the torch with flint to enter dark places safely
    * Tie rope to cross a broken bridge
    * Find & use the iron key to open the ancient gate
    * Place the moon idol on the altar to win
- Win state: Restore the shrine
- Lose states: Fall into a pit in the dark, or get swept from the bridge without a rope
"""

from textwrap import fill

# -------------------------
# World definition
# -------------------------
world = {
    "locations": {
        "clearing": {
            "name": "Forest Clearing",
            "desc": "A round clearing bathed in pale moonlight. Paths lead north to a ravine and east into thicker woods.",
            "exits": {"north": "ravine_edge", "east": "thicket"},
            "items": ["torch", "note"]
        },
        "thicket": {
            "name": "Thicket",
            "desc": "Bristling shrubs and twisted limbs. You spot a coil of rope hanging from a branch.",
            "exits": {"west": "clearing", "south": "river_bank"},
            "items": ["rope"]
        },
        "river_bank": {
            "name": "River Bank",
            "desc": "Moonlight ripples across fast water. An old stone bridge once crossed here, but its center is missing.",
            "exits": {"north": "thicket", "east": "bridge_east"},
            "items": [],
            "state": {"rope_bridge": False}
        },
        "bridge_east": {
            "name": "Bridge – East Abutment",
            "desc": "A broken span reaches halfway across. A stout post stands here; the ravine yawns below.",
            "exits": {"west": "river_bank", "east": "gate"},
            "items": [],
            "state": {"rope_tied": False}
        },
        "ravine_edge": {
            "name": "Ravine Edge",
            "desc": "Jagged rocks and a gusting draft. A narrow path leads west along the ravine to a hidden ledge.",
            "exits": {"south": "clearing", "west": "ledge"},
            "items": []
        },
        "ledge": {
            "name": "Hidden Ledge",
            "desc": "A cramped ledge under a dark overhang. Something glints in the gloom.",
            "exits": {"east": "ravine_edge"},
            "items": ["flint", "iron_key"],
            "state": {"dark": True, "pit": True}
        },
        "gate": {
            "name": "Ancient Gate",
            "desc": "A moon-carved stone gate blocks a courtyard beyond. A keyhole waits beneath the crescent relief.",
            "exits": {"west": "bridge_east", "east": "courtyard"},
            "items": [],
            "state": {"locked": True}
        },
        "courtyard": {
            "name": "Shrine Courtyard",
            "desc": "Broken columns guard a weathered altar. A cracked tablet rests nearby, etched with faint words.",
            "exits": {"west": "gate", "east": "shrine"},
            "items": ["tablet"]
        },
        "shrine": {
            "name": "Inner Shrine",
            "desc": "A silent chamber with a shallow bowl on an altar. The bowl is rimmed with silver crescents.",
            "exits": {"west": "courtyard"},
            "items": [],
            "state": {"idol_placed": False}
        }
    },
    "items": {
        "torch": {
            "name": "torch",
            "desc": "A resin-soaked torch. It will burn bright if lit.",
            "portable": True,
            "state": {"lit": False}
        },
        "rope": {
            "name": "rope",
            "desc": "Tough hemp rope, coiled neatly.",
            "portable": True
        },
        "flint": {
            "name": "flint",
            "desc": "A sharp shard of flint perfect for sparking.",
            "portable": True
        },
        "iron_key": {
            "name": "iron key",
            "desc": "A heavy iron key etched with a crescent.",
            "portable": True
        },
        "moon_idol": {
            "name": "moon idol",
            "desc": "A small silver idol shaped like a crescent moon.",
            "portable": True
        },
        "note": {
            "name": "note",
            "desc": "A scrap of parchment: ‘Moonlight reveals, fire averts the fall.’",
            "portable": True
        },
        "tablet": {
            "name": "stone tablet",
            "desc": "Etched text: ‘Return the moon to her bowl and darkness shall lift.’",
            "portable": False
        }
    }
}

# Place the moon idol secretly (you’ll need to discover it)
# (We’ll spawn it when the player opens the gate to encourage exploration.)
# You can change this if you prefer it hidden elsewhere:
hidden_items = {"gate_opens_spawns": "moon_idol"}

# -------------------------
# Game state
# -------------------------
state = {
    "here": "clearing",
    "inventory": [],
    "alive": True,
    "won": False,
    "score": 0,
    "turns": 0
}

# -------------------------
# Utility
# -------------------------
def say(text):
    print(fill(text, width=80))

def look(location_id=None):
    if location_id is None:
        location_id = state["here"]
    loc = world["locations"][location_id]
    say(f"\n{loc['name']}")
    say(loc["desc"])
    # Contextual warnings/help
    if location_id == "ledge" and loc["state"].get("dark", False):
        say("It is very dark here.")
    items = loc.get("items", [])
    if items:
        vis_names = []
        for it in items:
            vis_names.append(world["items"][it]["name"])
        say("You see: " + ", ".join(sorted(set(vis_names))) + ".")
    exits = ", ".join(sorted(loc["exits"].keys()))
    say(f"Exits: {exits}")

def move(direction):
    loc = world["locations"][state["here"]]
    exits = loc["exits"]
    if direction not in exits:
        say("You can’t go that way.")
        return
    dest = exits[direction]

    # Hazards before moving (bridge gap)
    if (state["here"], direction) == ("river_bank", "east"):
        # crossing toward bridge_east
        if not world["locations"]["river_bank"]["state"]["rope_bridge"]:
            say("The missing span foams below. Without a secured rope bridge, you slip and are swept away!")
            lose("You were carried off by the river.")
            return

    # Hazards in dark
    if dest == "ledge":
        # entering a dark ledge without lit torch can cause a fall
        if world["locations"]["ledge"]["state"]["dark"] and not is_lit():
            say("You step into the overhang blind and stumble. The rock gives way beneath you…")
            lose("You fall into the ravine.")
            return

    state["here"] = dest
    look()

def take(thing):
    loc = world["locations"][state["here"]]
    items_here = loc.get("items", [])
    # allow short names
    match = match_item(thing, items_here)
    if not match:
        say("You don’t see that here.")
        return
    item = world["items"][match]
    if not item.get("portable", True):
        say("You can’t take that.")
        return
    loc["items"].remove(match)
    state["inventory"].append(match)
    say(f"You take the {item['name']}.")

def drop(thing):
    match = match_item(thing, state["inventory"])
    if not match:
        say("You aren’t carrying that.")
        return
    world["locations"][state["here"]]["items"].append(match)
    state["inventory"].remove(match)
    say(f"You drop the {world['items'][match]['name']}.")

def inventory():
    inv = state["inventory"]
    if not inv:
        say("You’re empty-handed.")
        return
    names = [world["items"][it]["name"] for it in inv]
    say("You are carrying: " + ", ".join(sorted(names)) + ".")

def read(thing):
    # can read if carrying or present
    target = match_item(thing, state["inventory"]) or match_item(thing, world["locations"][state["here"]]["items"])
    if not target:
        say("You can’t read what you don’t have.")
        return
    it = world["items"][target]
    if target in ("note", "tablet"):
        say(it["desc"])
    else:
        say("There’s nothing to read on that.")

def light(thing):
    target = match_item(thing, state["inventory"])
    if not target:
        say("You’ll need it in hand to light it.")
        return
    if target != "torch":
        say("That won’t light.")
        return
    if "flint" not in state["inventory"]:
        say("You strike nothing. You probably need flint.")
        return
    world["items"]["torch"]["state"]["lit"] = True
    say("You spark the flint — the torch flares to life.")
    # torch reveals the ledge (no longer dark)
    world["locations"]["ledge"]["state"]["dark"] = False

def open_gate():
    gate = world["locations"]["gate"]
    if state["here"] != "gate":
        say("You’re not near the gate.")
        return
    if not gate["state"]["locked"]:
        say("The gate already stands open.")
        return
    if "iron_key" not in state["inventory"]:
        say("The keyhole rejects your effort. You’ll need a key.")
        return
    gate["state"]["locked"] = False
    say("The iron key turns with a heavy click. The ancient gate swings inward.")
    # spawn the idol in the courtyard once opened (a little reward/progression)
    idol = hidden_items.get("gate_opens_spawns")
    if idol and idol not in world["locations"]["courtyard"]["items"] and idol not in state["inventory"]:
        world["locations"]["courtyard"]["items"].append(idol)

def use(obj, preposition=None, target=None):
    here = state["here"]

    # Rope on post to create a rope bridge
    if obj == "rope" and target in ("post", "bridge", "span") and here == "bridge_east":
        if "rope" not in state["inventory"]:
            say("You don’t have a rope.")
            return
        if world["locations"]["bridge_east"]["state"]["rope_tied"]:
            say("The rope is already secured.")
            return
        world["locations"]["bridge_east"]["state"]["rope_tied"] = True
        world["locations"]["river_bank"]["state"]["rope_bridge"] = True
        say("You tie the rope to the stout post and secure the span. A makeshift rope bridge now holds.")
        return

    # Generic key on gate (alias of open)
    if obj in ("key", "iron key") and here == "gate":
        open_gate()
        return

    # Place idol on bowl in shrine
    if obj in ("idol", "moon idol") and here == "shrine":
        if "moon_idol" not in state["inventory"]:
            say("You don’t have the idol.")
            return
        world["locations"]["shrine"]["state"]["idol_placed"] = True
        state["inventory"].remove("moon_idol")
        say("You set the moon idol into the silver bowl. A soft glow spreads through the chamber.")
        win("The shrine is restored. Dawn will be kind to this forest.")
        return

    say("Nothing happens.")

def is_lit():
    return world["items"]["torch"]["state"].get("lit", False)

def match_item(text, pool):
    if not pool:
        return None
    text = (text or "").strip().lower()
    # simple matching by substring
    for it in pool:
        name = world["items"][it]["name"].lower()
        if text == it or text == name or text in name:
            return it
    return None

def lose(reason):
    state["alive"] = False
    say(f"\nYou lose. {reason}")

def win(reason):
    state["won"] = True
    say(f"\nYou win! {reason}")

# -------------------------
# Command parsing
# -------------------------
def parse(cmd):
    # normalize
    cmd = cmd.strip().lower()
    if not cmd:
        return ("", [])
    parts = cmd.split()
    verb = parts[0]
    rest = parts[1:]
    return (verb, rest)

def handle(cmd):
    verb, rest = parse(cmd)
    state["turns"] += 1

    if verb in ("q", "quit", "exit"):
        state["alive"] = False
        say("You give up the quest.")
        return
    if verb in ("l", "look"):
        look()
        return
    if verb in ("i", "inv", "inventory"):
        inventory()
        return
    if verb in ("go", "walk", "move"):
        if not rest:
            say("Go where?")
            return
        move(rest[0])
        return
    if verb in ("n","s","e","w","north","south","east","west","up","down"):
        # compass shorthand
        dir_alias = {"n":"north","s":"south","e":"east","w":"west"}
        move(dir_alias.get(verb, verb))
        return
    if verb in ("take", "get", "grab"):
        if not rest:
            say("Take what?")
            return
        take(" ".join(rest))
        return
    if verb == "drop":
        if not rest:
            say("Drop what?")
            return
        drop(" ".join(rest))
        return
    if verb == "read":
        if not rest:
            say("Read what?")
            return
        read(" ".join(rest))
        return
    if verb == "light":
        if not rest:
            say("Light what?")
            return
        light(" ".join(rest))
        return
    if verb == "open":
        if rest and rest[0] == "gate":
            open_gate()
            return
        say("Open what?")
        return
    if verb in ("use","tie","place"):
        if not rest:
            say(f"{verb.title()} what?")
            return
        # parse patterns like "use rope on post"
        obj = rest[0]
        prep = None
        target = None
        if len(rest) >= 3 and rest[1] in ("on","with","to","into"):
            prep = rest[1]
            target = " ".join(rest[2:])
        use(obj, prep, target)
        return
    if verb == "help":
        say("Commands: look/l, go <dir> (or n/s/e/w), take <item>, drop <item>, read <item>,")
        say("light <item>, open gate, use <obj> [on/with/to] <target>, inventory/i, quit")
        return

    say("I don’t understand that command. Type 'help' for options.")

# -------------------------
# Game loop
# -------------------------
def start():
    say("Welcome to THE MOONLIT SHRINE.\nType 'help' for commands.\n")
    look()
    while state["alive"] and not state["won"]:
        try:
            cmd = input("\n> ")
        except EOFError:
            cmd = "quit"
        handle(cmd)
    say("\nThanks for playing!")

# -------------------------
# Start
# -------------------------
start()
